# Sistemas inteligentes para respostas a perguntas médicas

Gyovana M. Moriyama (216190)

Rafael A. Matumoto (273085)

In [ ]:
!pip install -qU langchain_openai langchain-community faiss-cpu sentence-transformers openai datasets pydantic langgraph rank_bm25

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.5/387.5 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.

## ReAct

In [ ]:
import os
import datasets
import datetime
import re

from google.colab import userdata, drive

from tqdm import tqdm
from pydantic import BaseModel, Field
from typing import List, Optional, Literal

from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceBgeEmbeddings

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools.retriever import create_retriever_tool
from langchain_core.load import dumps, loads
from langchain_community.retrievers import BM25Retriever
from rank_bm25 import BM25Okapi

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

### MedQA-USMLE-4-options dataset

In [ ]:
data = datasets.load_dataset('GBaker/MedQA-USMLE-4-options', split='test')

In [ ]:
sampled_question = data[0]

### Load saved retrievers

In [ ]:
import pickle

with open('/content/drive/MyDrive/IA024A_Processamento_de_Linguagem_Natural/Projeto Final/projeto/entrega3/retriever/bm25_retriever.pkl', 'rb') as f:
    retriever_bm25 = pickle.load(f)

In [ ]:
print(retriever_bm25.invoke('meconium ileus'))

[Document(metadata={'ref': 'Surgery_Schwartz.txt'}, page_content='meconium ileus, the terminal ileum is filled with pellets of meconium. In patients with complicated meco-nium ileus, intraperitoneal calcifications form, producing an eggshell pattern on plain abdominal X-ray.Management.\u2002The treatment strategy depends on whether the patient has complicated or uncomplicated meconium ileus. Patients with uncomplicated meconium ileus can be Brunicardi_Ch39_p1705-p1758.indd   172612/02/19   11:26 AM 1727PEDIATRIC SURGERYCHAPTER 39Figure 39-17.\u2002Ladd procedure for'), Document(metadata={'ref': 'Biochemistry_Lippincott.txt'}, page_content='Diagnosis: Meconium ileus (obstruction of the ileum by meconium, the first stool produced by newborns) was confirmed by abdominal x-rays. About 98% of full-term newborns with meconium ileus have cystic fibrosis (CF). Diagnosis of CF was subsequently confirmed with a chloride sweat test.\n\nTreatment: The ileus was successfully treated nonsurgically. 

In [ ]:
# create a search tool from the bm25 retriever
tool_search_bm25 = create_retriever_tool(
    retriever=retriever_bm25,
    name='search_docs',
    description='Searchs the query in documents and returns the top-3 most relevant ones.',
)

### ReAct agent

In [ ]:
model = 'gpt-4o-mini'
model_temp = 0.5

In [ ]:
llm = ChatOpenAI(
    model=model,
    temperature=model_temp,
)

In [ ]:
# react prompt
react_prompt_template = '''
Solve the question answering task alternating between Question, Thought, Action, Input and Observation steps.
You only have access to the following tools: {tools}

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do. Break the problem down into subproblems and smaller steps and decide which action to take.
Action: the action to take, must be one of [{tool_names}]. If no action is needed, return your Final answer instead.
Action Input: the input to the action
Observation: the output of the action.
... (this Thought/Action/Action Input/Observation can repeat any number of times)
Final Answer: A/B/C/D

Begin!
Question: {question}
Options: {options}
Thought: {agent_scratchpad}
'''.strip()

In [ ]:
react_prompt = ChatPromptTemplate.from_template(react_prompt_template)

### ReAct with BM25 Okapi retriever

https://python.langchain.com/docs/how_to/serialization/

In [ ]:
agent = create_react_agent(
    llm=llm,
    tools=[tool_search_bm25],
    prompt=react_prompt
)

In [ ]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=[tool_search_bm25],
    verbose=False,
    handle_parsing_errors=True,
    return_intermediate_steps=True,
    max_iterations=5
)

In [ ]:
res = agent_executor.invoke({'question': sampled_question['question'], 'options': sampled_question['options']})

In [ ]:
res

{'question': 'A junior orthopaedic surgery resident is completing a carpal tunnel repair with the department chairman as the attending physician. During the case, the resident inadvertently cuts a flexor tendon. The tendon is repaired without complication. The attending tells the resident that the patient will do fine, and there is no need to report this minor complication that will not harm the patient, as he does not want to make the patient worry unnecessarily. He tells the resident to leave this complication out of the operative report. Which of the following is the correct next action for the resident to take?',
 'options': {'A': 'Disclose the error to the patient and put it in the operative report',
  'B': 'Tell the attending that he cannot fail to disclose this mistake',
  'C': 'Report the physician to the ethics committee',
  'D': 'Refuse to dictate the operative report'},
 'output': 'A',
 'intermediate_steps': [(AgentAction(tool='search_docs', tool_input="'ethical obligation t

In [ ]:
res['intermediate_steps']

[(AgentAction(tool='search_docs', tool_input="'ethical obligation to disclose surgical errors'", log="The situation involves an ethical dilemma regarding the disclosure of a surgical error. The resident must consider the principles of medical ethics, including honesty, transparency, and the duty to inform the patient about any complications that could affect their care. \n\n1. **Identify the key issues**: The resident made an error by cutting a flexor tendon, which was repaired without complication. The attending physician suggests not disclosing this in the operative report or to the patient. The resident must decide how to handle this situation ethically.\n\n2. **Evaluate the options**:\n   - **Option A**: Disclosing the error to the patient and including it in the operative report aligns with ethical standards of transparency and patient autonomy.\n   - **Option B**: Telling the attending that he cannot fail to disclose the mistake may create conflict but reinforces the ethical obli

In [ ]:
print(res['intermediate_steps'][0][0].log)

The situation involves an ethical dilemma regarding the disclosure of a surgical error. The resident must consider the principles of medical ethics, including honesty, transparency, and the duty to inform the patient about any complications that could affect their care. 

1. **Identify the key issues**: The resident made an error by cutting a flexor tendon, which was repaired without complication. The attending physician suggests not disclosing this in the operative report or to the patient. The resident must decide how to handle this situation ethically.

2. **Evaluate the options**:
   - **Option A**: Disclosing the error to the patient and including it in the operative report aligns with ethical standards of transparency and patient autonomy.
   - **Option B**: Telling the attending that he cannot fail to disclose the mistake may create conflict but reinforces the ethical obligation.
   - **Option C**: Reporting the physician to the ethics committee may be seen as an extreme action 

In [ ]:
print(res['intermediate_steps'][0][1])

media. However, such disclosures may be more difficult on social media due to the limited number of allowable characters (i.e., 140 characters for Twitter) or to the way information is propa-gated. For example, a surgeon may disclose an industry relation-ship on an original tweet, but the disclosure may not appear in subsequent comments of a discussion thread. Surgeons post-ing on social media must make it their ethical and professional obligation to disclose their conflicts of interest.

As more clinical experience is acquired, ET appears more benign than previously thought. Evolution to acute leukemia is more likely to be a consequence of therapy than of the disease itself. In managing patients with thrombocytosis, the physician’s first obligation is to do no harm.

Confidentiality: a person’s right to decide how and to whom personal medical information will be communicated is part of autonomy

Beneficence: the obligation to promote the well-being of others or, in medicine, to benefi

In [ ]:
res['output']

'A'

In [ ]:
with open(f'/content/drive/MyDrive/IA024A_Processamento_de_Linguagem_Natural/Projeto Final/projeto/entrega3/results/answers_bm25_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}.txt', 'w') as f:

    f.write(''.center(10, '-'))
    f.write('\n')
    f.write(f'Model: {model}\n')
    f.write(f'Temperature: {model_temp}\n')
    f.write(f'Prompt: {react_prompt}\n')
    f.write(''.center(10, '-'))
    f.write('\n[{')

    for n, question in enumerate(tqdm(data)):

        res = agent_executor.invoke({'question': question['question'], 'options': question['options']})
        f.write(f'"{n}": ')

        # serialize results as a string
        f.write(dumps(res))
        f.write(',')

    f.write('"placeholder": "placeholder"') # add a placeholder k:v pair for trailling comma above
    f.write('}]')

100%|██████████| 1273/1273 [2:42:08<00:00,  7.64s/it]


#### Results BM25

In [ ]:
os.listdir('/content/drive/MyDrive/IA024A_Processamento_de_Linguagem_Natural/Projeto Final/projeto/entrega3/results')

['answers_faiss_20241103_151837.txt',
 'answers_faiss_20241103_152229.txt',
 'answers_faiss_20241103_152940.txt',
 'answers_faiss_20241103_153308.txt',
 'answers_faiss_20241103_153501.txt',
 'answers_faiss_20241103_154054.txt',
 'answers_faiss_20241103_155219.txt',
 'answers_faiss_20241103_165217.txt',
 'dataset_faiss_500_50',
 'dataset_faiss_500_50.csv',
 'answers_bm25_20241104_000018.txt',
 'dataset_bm25_500_50']

In [ ]:
# load log file with LLM results
with open(f'/content/drive/MyDrive/IA024A_Processamento_de_Linguagem_Natural/Projeto Final//projeto/entrega3/results/answers_bm25_20241104_000018.txt') as f:
    tmp = f.readlines()

In [ ]:
# recover results as a dictionary
recovered_results = loads(tmp[-1])[0]

<ipython-input-9-3475d7fc15ed>:2: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  recovered_results = loads(tmp[-1])[0]


In [ ]:
# 1273 questions + placeholder key
len(recovered_results)

1274

In [ ]:
# check outputs
for n, (k, v) in enumerate(recovered_results.items()):

    if k == 'placeholder':
        continue

    print(k, v['output'], data[n]['answer_idx'])

0 A B
1 D D
2 B B
3 D D
4 B B
5 B D
6 B C
7 C C
8 B B
9 Agent stopped due to iteration limit or time limit. A
10 D D
11 D D
12 B B
13 D D
14 Agent stopped due to iteration limit or time limit. C
15 B B
16 D D
17 D D
18 B B
19 B D
20 C C
21 A A
22 C C
23 A D
24 A A
25 D D
26 D D
27 A A
28 D D
29 C C
30 D D
31 Agent stopped due to iteration limit or time limit. C
32 D D
33 D B
34 D A
35 A A
36 D D
37 A A
38 C D
39 A A
40 C C
41 C C
42 A A
43 A B
44 C A
45 D (Fomepizole) D
46 C C
47 Agent stopped due to iteration limit or time limit. A
48 B B
49 C C
50 D D
51 A A
52 D D
53 Agent stopped due to iteration limit or time limit. C
54 A A
55 A A
56 Agent stopped due to iteration limit or time limit. C
57 C B
58 C C
59 C A
60 B C
61 A A
62 Agent stopped due to iteration limit or time limit. D
63 A A
64 A A
65 B B
66 C C
67 B B
68 D D
69 Agent stopped due to iteration limit or time limit. A
70 A A
71 A A
72 Agent stopped due to iteration limit or time limit. C
73 Agent stopped due to iteration li

In [ ]:
limit_error = list()
not_parsed = list()
answers = list()

# matches literals A, B, C, or D at the beginning of a sentence (followed by the text of the corresponding option)
# e.g. A (answer text) => A
# other cases will be ignored
ans_ptn = re.compile(r'(^[A|B|C|D])\s\(')

for n, (k, v) in enumerate(recovered_results.items()):

    # skip placeholder key
    if k == 'placeholder':
        continue

    # max_iteration set to 5
    if v['output'] == 'Agent stopped due to iteration limit or time limit.':
        answers.append(None)
        limit_error.append(k)

    elif len(v['output']) > 1:
        tmp_output = ans_ptn.findall(v['output'])
        if len(tmp_output) == 1:
            answers.append(tmp_output[0])
        elif len(tmp_output) > 1:
            answers.append(None)
            not_parsed.append(k)
            print(tmp_output)
        else:
            answers.append(None)
            not_parsed.append(k)
            print(k, v['output'])

    else:
        if v['output'] in ['A', 'B', 'C', 'D']:
            answers.append(v['output'])
        else:
            answers.append(None)

817 A/C
1077 A** (Erythroblasts with mitochondrial iron deposits).


In [ ]:
# unanswered questions or answers not properly parsed
len(limit_error)

74

In [ ]:
# parsing errors/other errors
len(not_parsed)

2

In [ ]:
# result_dict[question]['intermediate_steps'] -> List[List[AgentAction(tool, tool_input, log), response: str]]
def format_intermediate_step(answer):

    formatted_output = ''
    for i in answer['intermediate_steps']:
        formatted_output += i[0].log
        formatted_output += '\n\n'
        formatted_output += i[1]
        formatted_output += '\n'

    return formatted_output

In [ ]:
react_logs = list()

for i in recovered_results.values():
    if i == 'placeholder':
        continue
    react_logs.append(format_intermediate_step(i))

In [ ]:
data = data.add_column('answer_react_bm25', answers)
data = data.add_column('react_log', react_logs)

In [ ]:
data.save_to_disk(f'/content/drive/MyDrive/IA024A_Processamento_de_Linguagem_Natural/Projeto Final/projeto/entrega3/results/dataset_bm25_500_50')

Saving the dataset (0/1 shards):   0%|          | 0/1273 [00:00<?, ? examples/s]

In [ ]:
#Accuracy
sum([pred == gt for gt, pred in zip(data['answer_idx'], answers)]) / len(data)

0.7242733699921445

In [ ]:
df = data.to_pandas()

In [ ]:
df.to_csv(f'/content/drive/MyDrive/IA024A_Processamento_de_Linguagem_Natural/Projeto Final/projeto/entrega3/results/dataset_bm25_500_50.csv')

In [ ]:
# react with bm25 accuracy
accuracy_cot = df['answer_react_bm25'].eq(df['answer_idx']).mean()
accuracy_cot

0.7242733699921445

In [ ]:
# react with bm25 accuracy for step 1 questions
step1 = df[df['meta_info'].eq('step1')]

step1_acc = step1['answer_react_bm25'].eq(step1['answer_idx']).mean()
step1_acc

0.711340206185567

In [ ]:
# react with bm25 accuracy for step 2&3 questions
step23 = df[df['meta_info'].eq('step2&3')]

step23_acc = step23['answer_react_bm25'].eq(step23['answer_idx']).mean()
step23_acc

0.7390572390572391

In [ ]:
#Correct answers
df[df['answer_react_bm25'].eq(df['answer_idx'])]

,question,answer,options,meta_info,answer_idx,metamap_phrases,answer_react_bm25,react_log
1,A 67-year-old man with transitional cell carci...,Cross-linking of DNA,"{'A': 'Inhibition of proteasome', 'B': 'Hypers...",step1,D,"[67 year old man, transitional cell carcinoma ...",D,"To answer the question, I need to analyze the ..."
2,Two weeks after undergoing an emergency cardia...,Cholesterol embolization,"{'A': 'Renal papillary necrosis', 'B': 'Choles...",step2&3,B,"[Two weeks, emergency cardiac, stenting, unsta...",B,"To address the question, I need to analyze the..."
3,A 39-year-old woman is brought to the emergenc...,"Lactose-fermenting, gram-negative rods forming...","{'A': 'Coagulase-positive, gram-positive cocci...",step1,D,"[year old woman, brought, emergency department...",D,"To answer the question, I need to analyze the ..."
4,A 35-year-old man comes to the physician becau...,Ketotifen eye drops,"{'A': 'Erythromycin ointment', 'B': 'Ketotifen...",step2&3,B,"[35 year old man, physician, of itchy, watery,...",B,"To solve this question, I need to analyze the ..."
7,A 65-year-old man is brought to the emergency ...,Clopidogrel,"{'A': 'Nifedipine', 'B': 'Enoxaparin', 'C': 'C...",step2&3,C,"[65 year old man, brought, emergency departmen...",C,To determine the most appropriate medication t...
...,...,...,...,...,...,...,...,...
1264,A 40-year-old woman with Down syndrome comes t...,Surgical fusion of C1/C2,"{'A': 'Administration of botulism antitoxin', ...",step2&3,B,"[40 year old woman, Down syndrome, physician, ...",B,"To solve this question, I need to analyze the ..."
1267,A 29-year-old woman comes to the physician wit...,Plasmodium falciparum,"{'A': 'Plasmodium ovale', 'B': 'Plasmodium fal...",step2&3,B,"[29 year old woman, physician, high-grade feve...",B,"To solve this question, I need to analyze the ..."
1268,A 39-year-old woman presents to the clinic for...,Thyroid-stimulating hormone (TSH),"{'A': 'Glucose', 'B': 'Triiodothyronine (T3)',...",step2&3,D,"[year old woman presents, clinic, annual physi...",D,"To answer the question, I need to analyze the ..."
1271,A 19-year-old man is brought to the physician ...,Schizotypal personality disorder,"{'A': 'Social anxiety disorder', 'B': 'Avoidan...",step2&3,C,"[year old man, brought, physician, mother, wor...",C,To determine the most likely diagnosis for the...


In [ ]:
# distribution of wrong answers
df[~df['answer_react_bm25'].eq(df['answer_idx'])].groupby('answer_react_bm25').size()

,0
answer_react_bm25,
A,68
B,67
C,72
D,68


In [ ]:
# sample of correctly answered question
sample_answer = df[df['answer_react_bm25'].eq(df['answer_idx'])].sample(1)

In [ ]:
print(sample_answer['question'].values[0])

A 23-year-old woman presents to the hospital for elective surgery. However, due to an unexpected bleeding event, the physician had to order a blood transfusion to replace the blood lost in the surgery. After this, the patient became irritable and had difficulty breathing, suggesting an allergic reaction, which was immediately treated with epinephrine. This patient is otherwise healthy, and her history does not indicate any health conditions or known allergies. The physician suspects an immunodeficiency disorder that was not previously diagnosed. If serum is taken from this patient to analyze her condition further, which of the following would be expected?


In [ ]:
print(sample_answer['options'].values[0])

{'A': 'High IgM, low IgA, and IgE', 'B': 'Low IgA, normal IgG, and IgM', 'C': 'High IgE, normal IgA, and IgG', 'D': 'Normal serum immunoglobulin values'}


In [ ]:
print(sample_answer['answer_react_bm25'].values[0])

B


In [ ]:
print(sample_answer['react_log'].values[0])

To solve the question, I need to analyze the clinical scenario presented. The patient experienced an allergic reaction after a blood transfusion, which suggests an issue with her immune response. The physician suspects an immunodeficiency disorder, which could affect her immunoglobulin levels. 

Next, I need to consider the options provided and correlate them with common immunodeficiency disorders. 

1. **Option A (High IgM, low IgA, and IgE)**: This profile could suggest a specific type of immunodeficiency, but it's not typical.
2. **Option B (Low IgA, normal IgG, and IgM)**: This could indicate a specific immunodeficiency related to IgA deficiency, which is common and can lead to increased susceptibility to infections.
3. **Option C (High IgE, normal IgA, and IgG)**: This is often seen in allergic reactions but may not align with a primary immunodeficiency.
4. **Option D (Normal serum immunoglobulin values)**: This would suggest that there is no immunodeficiency, which contradicts th